In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [2]:
from load_functions import load_text
    
text_input = load_text(day=3)
text_input[:5]

['000110000001',
 '011011001101',
 '001101100111',
 '001101011001',
 '110111011101']

Get inputs as list of individual numbers

In [4]:
num_list = [list(t) for t in text_input]
num_list[:5]

[['0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1'],
 ['0', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0', '1'],
 ['0', '0', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1'],
 ['0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1'],
 ['1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1']]

# Problem 

#### Part 1: find most common bits at each position

In [73]:
# Get length of each binary sequence
bin_len = len(text_input[0])

# Initialise empty dict
from collections import defaultdict
d_dict = defaultdict()
for bin_pos in range(bin_len):
    d_dict[bin_pos] = 0

# Loop through each input and get sum at each position
for t in text_input:
    for bin_pos in range(bin_len):
        d_dict[bin_pos] += int(t[bin_pos])

# Then divide by length of inputs
for k in d_dict.keys():
    d_dict[k] = d_dict[k] / len(text_input)

# And get binary outputs
gamma_rate_bin = ''.join(['1' if d_dict[k] > 0.5 else '0' for k in d_dict.keys()])
epsilon_rate_bin = ''.join(['1' if d_dict[k] < 0.5 else '0' for k in d_dict.keys()])


# Convert to base 10
def bin_to_base_10(binary):
    
    result = 0
    
    # Start from 0 index and work upwards
    rev_binary_str = list(reversed(list(str(binary))))
    
    for i in range(len(rev_binary_str)):
        if rev_binary_str[i] == '1':
            result += 2**i
            
    return result
    
gamma_rate = bin_to_base_10(gamma_rate_bin)
epsilon_rate = bin_to_base_10(epsilon_rate_bin)

print('Day 3, part 1:', gamma_rate * epsilon_rate)

Day 3, part 1: 3429254


#### Part 2: after finding the most common bit in a position, filter to only keep numbers with that bit in that position

In [80]:
import numpy as np
import pandas as pd

df_inputs = pd.DataFrame(num_list).astype(int)

def filter_on_criteria(df, criteria, tie_breaker_digit):
    
    df_output = df.copy(deep=True)
    
    for col in df_output.columns:
    
        # Get row count
        df_len = df_output.shape[0]
        if df_len == 1:
            # Can't filter further
            return df_output

        # Sum columns to get number of 1's
        n_ones = df_output[col].sum()

        # Then get pct that are 1's
        pct_ones = n_ones / df_len

        # And get most common digit (handle tie breaks later on)
        most_common_digit = 0 if pct_ones <= 0.5 else 1

        # Filter on criteria
        if pct_ones == 0.5:
            df_output = df_output[df_output[col] == tie_breaker_digit].copy(deep=True)
        elif criteria == 'most':
            df_output = df_output[df_output[col] == most_common_digit].copy(deep=True)
        elif criteria == 'least':
            df_output = df_output[df_output[col] != most_common_digit].copy(deep=True)
        else:
            return 'Unhandled criteria'
    
    return df_output


# Then run this for 2 criteria
df_oxygen = filter_on_criteria(df_inputs, 'most', 1)
df_co2 = filter_on_criteria(df_inputs, 'least', 0)


# Convert to integer values
oxygen_rating = bin_to_base_10(''.join([str(s) for s in df_oxygen.iloc[0].values]))
co2_rating = bin_to_base_10(''.join([str(s) for s in df_co2.iloc[0].values]))
        
print('Day 3, part 2:', oxygen_rating * co2_rating)

Day 3, part 2: 5410338
